In [1]:
#use this command to install the required packages 
!pip install -r requirements.txt 

ERROR: Invalid requirement: '-'


In [1]:
#install these packages to get the resources you need
import pandas as pd
import numpy as np
import requests
import dask as dd

In [2]:
%%time 
#here is the link to the file in Google Drive

url_raw = 'https://drive.google.com/file/d/13vj4YxKRs1Tw1VPGhTefBxxKQuC5pRd_/view?usp=sharing'
url_raw_mn = 'https://drive.google.com/file/d/1YRh58UiKWNtxfvYa0IzFF3oYL42iBs7d/view?usp=sharing'

req = requests.get(url_raw)
url_content = req.content
with open('test_data.csv',) as file:
    df=pd.read_csv(file,delimiter=',')

req_mn = requests.get(url_raw_mn)
url_content_mn = req_mn.content
with open('test_data_mn.csv',) as file_mn:
    df_mn=pd.read_csv(file_mn,delimiter=',')

CPU times: user 1min 39s, sys: 2min 23s, total: 4min 2s
Wall time: 6min 14s


In [4]:
#look at the properties of the two datasets 
print(df.head())
print(df.shape)


# print(df_mn.head())
# print(df_mn.shape)

   vb_voterbase_id vb_voterbase_household_id vb_tsmart_exact_address_track  \
0  AK-000000000020      Y000000186098323KYLE              Y000000186098323   
1  AK-000000000250    Y000000182911889OSWALD              Y000000182911889   
2  AK-000000000509    Y000000235819126HOLMES              Y000000235819126   
3  AK-000000000711     Y000000140159129SCOTT              Y000000140159129   
4  AK-000000001144    Y000000084251981LARSEN              Y000000084251981   

   vb_tsmart_latitude  vb_tsmart_longitude  vb_tsmart_zip vb_tsmart_state  \
0           61.095810          -149.828888          99516              AK   
1           61.184738          -149.751389          99504              AK   
2           48.383949          -115.551353          59923              MT   
3           61.319233          -149.532120          99577              AK   
4           61.423569          -149.480682          99567              AK   

  vb_tsmart_level      vb_tsmart_county_name  vb_tsmart_county_code 

In [5]:
df.columns

Index(['vb_voterbase_id', 'vb_voterbase_household_id',
       'vb_tsmart_exact_address_track', 'vb_tsmart_latitude',
       'vb_tsmart_longitude', 'vb_tsmart_zip', 'vb_tsmart_state',
       'vb_tsmart_level', 'vb_tsmart_county_name', 'vb_tsmart_county_code',
       'add_type', 'tsmart_geo_confidence', 'vb_voterbase_gender',
       'vb_voterbase_race', 'vb_education', 'tb_education_cd',
       'tb_college_age_flg', 'tb_starting_out_flg', 'tb_career_bldr_flg',
       'tb_prime_earner_flg', 'tb_retiree_flg', 'vb_occupation',
       'tb_occupation_cd', 'xpg_ind_lvl_occupation_code',
       'xpg_ind_lvl_occupation_group', 'hh_income', 'tb_income_range_prem_cd',
       'xpg_ind_lvl_business_owner', 'vb_business_owner_indicator',
       'vb_business_size_code', 'vb_business_type',
       'vb_business_revenue_range', 'tb_business_owner_flg', 'tb_boah_flg',
       'tb_boah_biz_size_cd', 'tb_boah_biz_source_cd', 'tb_boah_biz_rank_cd',
       'tb_boah_biz_func_cd', 'tb_boah_biz_revenue_cd', 'tb_b